# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# загрузим всё необходимое
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
# загрузим данные
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
except:
    df0 = pd.read_csv('geo_data_0.csv')
try:
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
except:
    df1 = pd.read_csv('geo_data_1.csv')
try:
    df2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df2 = pd.read_csv('geo_data_2.csv')

In [3]:
# посмотрим на них
names= [df0, df1, df2]

def info_func(names):
    for name in names:
        df_temp = pd.DataFrame(name)
        display(df_temp.head(10))
        display(df_temp.info())
        display('Кол-во дублей:', df_temp.duplicated().sum())
        display('Кол-во НАНов:', df_temp.isna().sum())
    return
info_func(names)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

'Кол-во дублей:'

0

...

None

'Кол-во дублей:'

0

'Кол-во НАНов:'

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Вроде прилично всё, можно приступать к делению на обучающую и валидационную выборки. Целевой признак - product

In [4]:
target0 = df0['product']
target1 = df1['product']
target2 = df2['product']
# id нам вряд-ли понадобится в этой задаче, поэтому его тоже отправим на покой
features0 = df0.drop(['id', 'product'] , axis=1) # id нам вряд-ли понадобится в этой задаче, поэтому его тоже отправим на покой
features1 = df1.drop(['id', 'product'] , axis=1)
features2 = df2.drop(['id', 'product'] , axis=1)
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=12345)
features1_train, features1_valid, target1_train, target1_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345)
features2_train, features2_valid, target2_train, target2_valid = train_test_split(
    features2, target2, test_size=0.25, random_state=12345)

Данные подготовили, приступаем к обучению

## Обучение и проверка модели

Обучим модель линейной регрессии для каждого региона, которая будет предсказывать производительность скважин

In [5]:
model = LinearRegression()
# Зона 0
model.fit(features0_train, target0_train)
predicted0_valid = pd.Series(model.predict(features0_valid))
mse0 = mean_squared_error(target0_valid, predicted0_valid)
# Зона 1
model.fit(features1_train, target1_train)
predicted1_valid = pd.Series(model.predict(features1_valid))
mse1 = mean_squared_error(target1_valid, predicted1_valid)
# Зона 2
model.fit(features2_train, target2_train)
predicted2_valid = pd.Series(model.predict(features2_valid))
mse2 = mean_squared_error(target2_valid, predicted2_valid)

print("Значения модели")
print("Средний запас сырья в скважине =", predicted0_valid.mean(), predicted1_valid.mean(), predicted2_valid.mean())
print("RMSE =", mse0 ** 0.5, mse1 ** 0.5, mse2 ** 0.5)



Значения модели
Средний запас сырья в скважине = 92.59256778438035 68.72854689544602 94.96504596800489
RMSE = 37.5794217150813 0.8930992867756165 40.02970873393434


Кажется, что 0 и 2 зоны - самые перспективные для разработки, однако, у них и самая высокая метрика RMSE. Зато, 1 - самая стабильная, а значит, попасть в хорошую скважину там легче.

## Подготовка к расчёту прибыли

А что там с прибылью? Нужно разобраться

In [6]:
# По условию задачи имеем следующие параметры для расчёта
price = 450 * 1000 / 1000000 # цена 1000 баррелей нефти, млн. руб.
budget = 10000000000 / 1000000 # бюджет на разработку скважин, млн. руб.
n = 500 # количество скважин в каждой гео-зоне
n_max = 200 # максимальное количество скважин в каждой гео-зоне, которые будут разрабатывать
loss_prob = 0.025 # порог приемлемой вероятности убытков
bootstrap_samples = 1000
product_min = budget / (price * n_max)
display("Достаточный объём сырья для безубыточной разработки новой скважины", product_min)

'Достаточный объём сырья для безубыточной разработки новой скважины'

111.11111111111111

Таким образом, нам подойдут только те регионы, где, в среднем с 200 скважин, можно будет получить 111.1(1) баррелей нефти. Это довольно сильно разнится с расчётными данными из предыдущего пункта, однако, там у нас 500 скважин с разным объёмом. Возможно мы сможем отобрать в каждом регионе 200 скважин с требуемыми параметрами (в 1 скорее всего тоже сможем, скорее всего, но пока что это не очевидно)

## Расчёт прибыли и рисков 

### Расчёт прибыли 200 самых привлекательных скважин

In [7]:
def income(target_valid, predicted_valid):
    pred_choice = predicted_valid.sort_values(ascending=False).head(200) # выбираем 200 скважин с максимальным объемом нефти
    target_choice = target_valid.reset_index(drop=True)[pred_choice.index] # определяем верные значения для отобранных 200 скважин
    income = target_choice.sum() * price # с 200 скважин
    return income

In [8]:
print('Прибыль с 200 выбранных по предсказаниям модели скважин составит:')
print('для региона 0 - {:.3f} млн. руб.'.format(income(target0_valid, predicted0_valid) - budget))
print('для региона 1 - {:.3f} млн. руб.'.format(income(target1_valid, predicted1_valid) - budget))
print('для региона 2 - {:.3f} млн. руб.'.format(income(target2_valid, predicted2_valid) - budget))

Прибыль с 200 выбранных по предсказаниям модели скважин составит:
для региона 0 - 3320.826 млн. руб.
для региона 1 - 2415.087 млн. руб.
для региона 2 - 2710.350 млн. руб.


### Расчет рисков и прибыли для каждого региона (для 500 случайных скважин)

Чтобы расчитать выручку в случае, когда задается количество скважин, по которым производится поиск наибольших значений запасов, функцию нужно слегка доработать

In [9]:
def income1(target_valid, predicted_valid, n_max):
    pred_choice = predicted_valid.sort_values(ascending=False) # выбираем 200 скважин с максимальным объемом нефти
    target_choice = target_valid[pred_choice.index][:n_max] # определяем верные значения для отобранных 200 скважин
    income = target_choice.sum() * price # с 200 скважин
    return income

Далее, с помощью техники Bootstrap, напишем функцию, которая будет считать выручку и оценивать не убыточен ли регион. По итогу 1000 циклов получим распределение выручек, из которых определим среднюю выручку для региона, доверительный интервал и вероятность, получить отрицательную выручку.

In [10]:
def income_distribution (predicted_valid, target_valid, bootstrap_samples, n):
    state = np.random.RandomState(12345)
    count = 0 # с помощью этого счетчика будем считать вероятность убытков в регионе
    values = [] # здесь будут все значения выручек для 1000 выборок
    for i in range(bootstrap_samples): # создаем 1000 выборок
    
        # у целевого признака валидац. выборки сбрасываем индексы и случайно выбираем нужное количество скважин
        target_subsample = target_valid.reset_index(drop=True).sample(n, replace=True, random_state=state)
        
        # из предсказаний выбираем строки соответствующие отобранным строкам в целевом признаке
        probs_subsample = predicted_valid[target_subsample.index]
        
        # считаем выручку для отобранных строк, но только для нужного количества скважин
        rev = income1(target_subsample, probs_subsample, n_max) - budget
        # добавляем выручку в список выручек всех выборок
        values.append(rev)
        
        #проверяем не является выборка убыточной, если является, то увеличиваем счетчик
        if rev < 0:
            count +=1

    values = pd.Series(values)
    
    #посчитаем и выведем среднюю выручку
    mean = values.mean()
    print("Средняя выручка:", mean, 'млн рублей')
    
    # посчитаем и выведем 95-% доверительный интервал
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), np.std(values, ddof=1))
    print("95%-ый доверительный интервал:", confidence_interval)
    
    pvalue = 1. * count / bootstrap_samples
    if pvalue < loss_prob:
        print("Вероятность убытков равна {:.2%}, регион подходит по критериям".format(pvalue))
    else:
        print("Вероятность убытков равна {:.2%}, регион не подходит по критериям".format(pvalue))

In [11]:
print('Для первого региона при случайном выборе 500 скважин получаются следующие показатели')
income_distribution (predicted0_valid, target0_valid, bootstrap_samples, n)

Для первого региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 425.9385269105927 млн рублей
95%-ый доверительный интервал: (-118.17308158673279, 970.0501354079182)
Вероятность убытков равна 6.00%, регион не подходит по критериям


In [12]:
print('Для первого региона при случайном выборе 500 скважин получаются следующие показатели')
income_distribution (predicted1_valid, target1_valid, bootstrap_samples, n)

Для первого региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 515.2227734432902 млн рублей
95%-ый доверительный интервал: (85.11198847686433, 945.333558409716)
Вероятность убытков равна 1.00%, регион подходит по критериям


In [13]:
print('Для первого региона при случайном выборе 500 скважин получаются следующие показатели')
income_distribution (predicted2_valid, target2_valid, bootstrap_samples, n)

Для первого региона при случайном выборе 500 скважин получаются следующие показатели
Средняя выручка: 435.00836278275585 млн рублей
95%-ый доверительный интервал: (-120.12349557730545, 990.1402211428172)
Вероятность убытков равна 6.40%, регион не подходит по критериям


## Выводы

* Согласно расчитанной потенциальной средней выручке все три региона являются прибыльными, средние доходы с одной скважины приняли значения: регион 0 - 425,93 млн. рублей, регион 1 - 515,22 млн. рублей, регион 2 - 435,01  млн. рублей.
* Однако, 0-ый и 2-й регионы обладают очень высокой вероятностью убытков (6 и 6.4 % соответственно), поэтому не проходят по критерию порога рисков (вероятность убытков должна быть меньше 2,5 %). Также эта ситуация хорошо описывается 95-% доврительным интервалом. Левая граница, которая отделяет 2.5 % данных для 1-го и 3-го регионов принимает отрицательное значение, что говорит о том, что более 2.5 % выборок имеют отрицательную прибыль.
* 1-й регион оказался лидирующим по всем показателям. Он обладает наибольшим средним доходом одной скважины, а также единственный соответствует условию по минимальной вероятности убытков (левая граница 95-% доверительного интервала - положительная). Для установки своей бензоколоники с минимальными рисками следует использовать только 1-й регион